In [1]:
import copy
import json
import math
import numpy as np
import pprint as pp
import random
import indyconfig
import indyjson

### Connect to Indy by IndyDCP

In [2]:
import indydcp_client as client

In [3]:
bind_ip   = "192.168.3.5"
server_ip = "192.168.3.105"
name = "NRMK-Indy7"
indy = client.IndyDCPClient(bind_ip, server_ip, name)
indy.connect()

Connect: Bind IP (192.168.3.130), Server IP (192.168.3.129)


True

In [4]:
indy.go_home()
# indy.go_zero()
# indy.get_joint_pos()

### Set Configuration

In [5]:
# Set Workspace Configuration
# 따로 object space, target space 지정해 주지 않으면, 같은 공간으로 설정됨
workspace = indyconfig.WorkSpace()

# Set Object Configuration
# STACK_TASK = 0
# PALLETIZE_TASK = 1
obj_config = indyconfig.TaskObjConfig(workspace,indyconfig.STACK_TASK, obj_num = 3, class_num = 2)

# command_config 객체 생성
command_config = indyjson.CommandConfig()

# json 파일 생성하는 객체 생성
JsonPro = indyjson.IndyMakeJsonProgram()

In [6]:
pp.pprint(obj_config.obj)
pp.pprint(obj_config.tar_pos)

[{'class_': 1, 'pos_': [0.4538226931408202, -0.29320397064796283, 0.46]},
 {'class_': 2, 'pos_': [0.41053120815335453, -0.23480499753115064, 0.46]},
 {'class_': 1, 'pos_': [0.48570094992972807, -0.2621793430292997, 0.46]}]
[[0.4538226931408202, -0.29320397064796283, 0.46],
 [0.41053120815335453, -0.23480499753115064, 0.46]]


### Stacking Task Logic

In [7]:
def move_command(obj_config, command_config, JsonPro, pos_xyz) :
    command_config.create_radnom_command_config()
    pos_xyz = copy.deepcopy(pos_xyz)
    pos_xyz.extend([0,180,0])
    command_config.p = pos_xyz
    ##joint 모드일 경우 q도 추가하는 구문 만들어야함 : 이때는 p->q로 만드는 함수 필요
    JsonPro.add_seq(command_config)


# pick & place json파일로 만드는 함수 구현 해야함!(IndyJsonMake함수에서 ㄱㄱㄱ)
def pick_command():
    pass

def place_command():
    pass

##singularity problem생기면? 어떡하지? frame move만 구현해뒀는데 joint move도 쓸 수 있게 구현
## 이경우에 blend가 의미가 있나?

for i, obj_ in enumerate(obj_config.obj) :
    for j in range(4):
        if j == 0 :# obj로 가기
            move_command(obj_config,command_config, JsonPro, obj_['pos_'])

        elif j == 1: #pick하기(놓고 다시 위치로 돌아오는 것 까지)
            #obj까지 내려가기
            pos_down = copy.deepcopy(obj_['pos_'])
            pos_down[-1] = indyconfig.OBJ_POS_Z+indyconfig.OBJ_SIZE_Z
            move_command(obj_config,command_config, JsonPro, pos_down )
            
            #pick
            pick_command()
            
            #다시 올라오기
            move_command(obj_config,command_config, JsonPro, obj_['pos_'] )
            
        elif j == 2: #target위치로 가기
            move_command(obj_config, command_config, JsonPro, obj_config.tar_pos[obj_['class_']-1] )
            
        else : #place하기
            #제일 높은 obj까지 내려가기
            n=0
            for k in range(i+1):
                if obj_['class_'] == obj_config.obj[k]['class_'] :
                    n +=1
            print(n)
            pos_down = copy.deepcopy(obj_config.tar_pos[obj_['class_']-1])
            pos_down[-1] = indyconfig.OBJ_POS_Z + (indyconfig.OBJ_SIZE_Z * n)
            move_command(obj_config, command_config, JsonPro, pos_down )
            
            #pick
            place_command()
            
            #다시 올라오기
            move_command(obj_config, command_config, JsonPro, obj_config.tar_pos[obj_['class_']-1] )
        print(command_config.p)

[0.4538226931408202, -0.29320397064796283, 0.46, 0, 180, 0]
[0.4538226931408202, -0.29320397064796283, 0.46, 0, 180, 0]
[0.4538226931408202, -0.29320397064796283, 0.46, 0, 180, 0]
1
[0.4538226931408202, -0.29320397064796283, 0.46, 0, 180, 0]
[0.41053120815335453, -0.23480499753115064, 0.46, 0, 180, 0]
[0.41053120815335453, -0.23480499753115064, 0.46, 0, 180, 0]
[0.41053120815335453, -0.23480499753115064, 0.46, 0, 180, 0]
1
[0.41053120815335453, -0.23480499753115064, 0.46, 0, 180, 0]
[0.48570094992972807, -0.2621793430292997, 0.46, 0, 180, 0]
[0.48570094992972807, -0.2621793430292997, 0.46, 0, 180, 0]
[0.4538226931408202, -0.29320397064796283, 0.46, 0, 180, 0]
2
[0.4538226931408202, -0.29320397064796283, 0.46, 0, 180, 0]


In [8]:
indy.set_and_start_json_program(json.dumps(JsonPro.JsonPro))

In [9]:
with open('stack_test.7.json', 'w') as fp:
    json.dump(JsonPro.JsonPro, fp)

KeyboardInterrupt: 

In [ ]:
indy.go_home()

##  Test

In [11]:
file_name = 'testtest.7.json'

with open(file_name) as json_file:
    json_data = json.load(json_file)
# pp.pprint(json_data)
testtest= json.dumps(json_data)
testtest

'{"program": [{"type": 999, "enable": true, "palletInfo": [], "toolInfo": [], "collisionPolicy": {"time": 2, "policy": 0}, "pId": 0, "id": 1}, {"type": 2, "varList": [], "enable": true, "pId": 0, "id": 2}, {"type": 103, "enable": true, "pId": 0, "name": "testtest", "id": 3}], "moveList": [{"tcp": [0, 0, 0, 0, 0, 0], "type": 103, "blendOpt": {"constant": false}, "boundary": {"accLevel": 3, "velLevel": 3}, "offset": {"pos": [0, 0, 0], "type": 0}, "refFrame": {"tref": [0, 0, 0, 0, 0, 0], "type": 1}, "wpList": [{"t": 2, "id": 0}], "intpl": 1, "name": "testtest"}], "wpList": [{"type": 0, "name": "", "p": [0.53541, 0.018198, 0.460146, 0, 180, 0], "tBase": 0, "q": [22.319669, -20.003301, -75.687677, 0, -84.309021, 22.319668], "stopBlend": true, "blendRadius": 0, "id": 0}]}'

In [12]:
indy.set_and_start_json_program(testtest)

In [15]:
p = [0.53541, 0.018198, 0.460146, 0, 180, 0]
q = [22.319669, -20.003301, -75.687677, 0, -84.309021, 22.319668]

In [16]:
command_config__= indyjson.CommandConfig(p= p, q = q, intpl = 1)
JsonPro__        = indyjson.IndyMakeJsonProgram()
JsonPro__.add_seq(command_config__)
make = json.dumps(JsonPro__.JsonPro)
make

'{"program": [{"type": 999, "enable": true, "palletInfo": [], "toolInfo": [], "collisionPolicy": {"time": 2, "policy": 0}, "pId": 0, "id": 1}, {"type": 2, "varList": [], "enable": true, "pId": 0, "id": 2}, {"type": 103, "enable": true, "pId": 0, "name": "Frame_move_to_VELO3.0_01", "id": 3}], "moveList": [{"tcp": [0, 0, 0, 0, 0, 0], "type": 103, "blendOpt": {"constant": false}, "boundary": {"accLevel": 3, "velLevel": 3}, "offset": {"pos": [0, 0, 0], "type": 0}, "refFrame": {"tref": [0, 0, 0, 0, 0, 0], "type": 1}, "wpList": [{"t": 2, "id": 0}], "intpl": 1, "name": "Frame_move_to_VELO3.0_01"}], "wpList": [{"type": 0, "name": "", "p": [0.53541, 0.018198, 0.460146, 0, 180, 0], "tBase": 0, "q": [22.319669, -20.003301, -75.687677, 0, -84.309021, 22.319668], "stopBlend": true, "blendRadius": 0, "id": 0}]}'

In [17]:
indy.set_and_start_json_program(make)